# Notebook 1: Why RAG? + Minimal Pipeline

**Why RAG:** The same question answered without context vs with context—without context the LLM may guess or hallucinate; with context the answer stays grounded in your data.

**Pipeline:** Load document → chunk → embed → store in ChromaDB → retrieve top-k → prompt LLM → answer.

**Next:** Notebook 2 runs several concrete RAG examples using the same utilities.

## Setup

In [1]:
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI()

CHAT_MODEL = "gpt-4o-mini"
TOP_K = 5  # we'll compare k=5 vs k=8 in the pipeline below

## Why RAG? Same question, two ways

We use a short excerpt as "retrieved context" and ask one question: first **without** context, then **with** context.

In [2]:
CONTEXT = """
AI Agent Insure offers a portfolio of AI-native insurance products structured around distinct AI risk domains:
- AI Infrastructure & Operations Protection
- Agentic AI Liability Insurance
- Autonomous Systems & Robotics Coverage
- Compliance & Regulatory Shield
- Agentic Workflow Uptime Insurance
- Intellectual Property & Output Protection
- Model & Data Security Insurance
- AI Incident Response & Crisis Management
- Synthetic Data & Dataset Integrity Coverage
""".strip()

question = "What products does AI Agent Insure offer?"

### Without context (raw LLM)

In [3]:
response_no = client.chat.completions.create(
    model=CHAT_MODEL,
    messages=[{"role": "user", "content": question}],
    temperature=0
)
print("Answer (no context):")
print(response_no.choices[0].message.content)

Answer (no context):
As of my last update in October 2023, AI Agent Insure is not a widely recognized company or brand in the insurance industry. It's possible that it could be a new or niche provider that has emerged since then, or it may refer to a specific product or service related to AI in the insurance sector.

If AI Agent Insure is a company that has launched recently, I recommend checking their official website or contacting them directly for the most accurate and up-to-date information regarding their products and services. Typically, insurance companies may offer products such as:

1. **Auto Insurance**
2. **Homeowners Insurance**
3. **Renters Insurance**
4. **Life Insurance**
5. **Health Insurance**
6. **Business Insurance**
7. **Travel Insurance**
8. **Disability Insurance**

If you have more specific information or context about AI Agent Insure, I would be happy to help further!


### With context (RAG-style)

Same **prompt template** for both demos; only the question changes. First: a question the context answers. Second: a question it doesn't.

In [4]:
def prompt_with_context(context: str, q: str) -> str:
    return f"""Use only the following context to answer the question. If the context doesn't contain the answer, say so.

Context:
{context}

Question: {q}

Answer:"""

# Demo 1: question the context answers
response_1 = client.chat.completions.create(
    model=CHAT_MODEL,
    messages=[{"role": "user", "content": prompt_with_context(CONTEXT, question)}],
    temperature=0
)
print("Question:", question)
print("Answer (with context):", response_1.choices[0].message.content)

Question: What products does AI Agent Insure offer?
Answer (with context): AI Agent Insure offers the following products:
- AI Infrastructure & Operations Protection
- Agentic AI Liability Insurance
- Autonomous Systems & Robotics Coverage
- Compliance & Regulatory Shield
- Agentic Workflow Uptime Insurance
- Intellectual Property & Output Protection
- Model & Data Security Insurance
- AI Incident Response & Crisis Management
- Synthetic Data & Dataset Integrity Coverage


In [5]:
# Demo 2: question the context does *not* answer (same template, different question)
new_question = "Who won the 2025 F1 Constructor's Championship?"
response_2 = client.chat.completions.create(
    model=CHAT_MODEL,
    messages=[{"role": "user", "content": prompt_with_context(CONTEXT, new_question)}],
    temperature=0
)
print("Question:", new_question)
print("Answer (with context):", response_2.choices[0].message.content)

Question: Who won the 2025 F1 Constructor's Championship?
Answer (with context): The context does not contain the answer to the question about the 2025 F1 Constructor's Championship.


Without context the model may guess or hallucinate. With context the answer is grounded in the document. **RAG** is the system that retrieves that context automatically.

## Minimal pipeline: load → chunk → embed → store → retrieve → generate

Same question, two retrieval settings: **top-5** vs **top-8**. With 5 chunks the one containing the product list often isn't in the retrieved set; with 8 it usually is. So the answer can go from "context doesn't contain the answer" to the full list.

In [6]:
from rag_utils import load_document, chunk_by_sentences, embed, build_index

doc = load_document()
chunks = chunk_by_sentences(doc)
print(f"Loaded {len(doc)} chars, {len(chunks)} chunks")

Loaded 4994 chars, 34 chunks


In [7]:
chroma_client, coll = build_index(chunks, collection_name="rag", client=client)
print("Index built.")

Index built.


In [8]:
def run_rag_query(coll, question: str, k: int, client):
    """Retrieve top-k chunks, build prompt with context, return (answer, retrieved_chunks)."""
    q_emb = embed([question], client=client)[0]
    results = coll.query(query_embeddings=[q_emb], n_results=k)
    retrieved = results["documents"][0]
    context = "\n".join(retrieved)
    prompt = prompt_with_context(context, question)
    response = client.chat.completions.create(
        model=CHAT_MODEL,
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    return response.choices[0].message.content, retrieved

In [9]:
# Top-5: chunk with product list often not in the retrieved set
answer_5, retrieved_5 = run_rag_query(coll, question, 5, client)
print("--- Top-5 retrieval ---")
print("Answer:", answer_5)

--- Top-5 retrieval ---
Answer: The context does not specify the exact products offered by AI Agent Insure. It mentions tailored insurance solutions addressing various risks related to AI systems, but does not list specific products.


In [10]:
# Top-8: product-list chunk usually included → full answer
answer_8, retrieved_8 = run_rag_query(coll, question, 8, client)
print("--- Top-8 retrieval ---")
print("Answer:", answer_8)

--- Top-8 retrieval ---
Answer: AI Agent Insure offers a portfolio of AI-native insurance products structured around distinct AI risk domains, including:

- AI Infrastructure & Operations Protection
- Agentic AI Liability Insurance
- Autonomous Systems & Robotics Coverage
- Compliance & Regulatory Shield
- Agentic Workflow Uptime Insurance
- Intellectual Property & Output Protection
- Model & Data Security Insurance
- AI Incident Response & Crisis Management
- Synthetic Data & Dataset Integrity Coverage


**Takeaway:** Retrieval quality (and **k**) directly affects whether the LLM can answer. Too small a k and the relevant chunk may be missed; larger k improves recall but adds noise and cost. Tune k (and chunking) for your data and queries.